## Bibliotecas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
import collections
import math

from timeit import default_timer as timer
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support , roc_auc_score, roc_curve, \
    auc, precision_score, recall_score, f1_score, accuracy_score
from statsmodels.tsa.tsatools import lagmat

%run ./config.ipynb

## Pós-processamento

In [2]:
def save_model(model, iterator, train_type, model_name=""):
    
    # Método 1
    model_json = model.to_json()
    with open(outputs_folder + train_type +  "_model_" + str(iterator) + "-" + model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(outputs_folder + train_type +  "_model_" + str(iterator) + ".h5")
    
    # Método 2
    model.save(outputs_folder + train_type +  "_model_" + str(iterator) + "-" + model_name + ".h5")
    
    # Salvando o history do treinamento em um CSV
    pd.DataFrame.from_dict(model.history.history).to_csv(outputs_folder + train_type +  "_model_" + str(iterator) + "-" + model_name + "-history.csv", sep=';', decimal=',')

## Métricas

In [ ]:
def calculate_roc_auc(x_data, y_data, model, model_name):
    
    # generate a no skill prediction (majority class)
    ns_probs = [0 for _ in range(len(x_data))]

    # predict probabilities
    lr_probs = model.predict_proba(x_data)

    # keep probabilities for the positive outcome only
    lr_probs = lr_probs[:, 1]

    # calculate scores
    try:
        ns_auc = roc_auc_score(y_data.argmax(axis=1), ns_probs)
        lr_auc = roc_auc_score(y_data.argmax(axis=1), lr_probs)
        
        # calculate roc curves
        ns_fpr, ns_tpr, _ = roc_curve(y_data.argmax(axis=1), ns_probs)
        lr_fpr, lr_tpr, _ = roc_curve(y_data.argmax(axis=1), lr_probs)
    except:
        ns_auc = roc_auc_score(y_data, ns_probs)
        lr_auc = roc_auc_score(y_data, lr_probs)
        
        # calculate roc curves
        ns_fpr, ns_tpr, _ = roc_curve(y_data, ns_probs)
        lr_fpr, lr_tpr, _ = roc_curve(y_data, lr_probs)

    # summarize scores
    print('No Skill: ROC AUC=%.3f' % (ns_auc))
    print(model_name + ': ROC AUC=%.3f' % (lr_auc))

    # plot the roc curve for the model
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    plt.plot(lr_fpr, lr_tpr, marker='.', label=model_name)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()

In [3]:
def metrics(y_real, y_pred, model, conf_matrix, STATUS, multi_problem):
    
    # Classificação multiclass
    if (multi_problem):
    
        cm_values = conf_matrix.values
    
        # Cálculo dos FP, FN e TP para cada classe
        tp = np.zeros((len(cm_values),1))
        tn = np.zeros((len(cm_values),1))
        fp = np.zeros((len(cm_values),1))
        fn = np.zeros((len(cm_values),1))
        acc = np.zeros((len(cm_values),1))

        for i in range(0, len(cm_values)):

            tp[i] = cm_values[i,i]
            fp[i] = cm_values[:,i].sum() - cm_values[i,i]
            fn[i] = cm_values[i,:].sum() - cm_values[i,i]
            tn[i] = len(y_real) - tp[i] - fp[i] - fn[i]
            acc[i] = (len(y_real) - (cm_values[i,:].sum() + cm_values[:,i].sum() - 2*cm_values[i,i]))/len(y_real)

        # Cálculo das métricas Precision, Recall e F-Score para cada classe
        try:
            metricas = precision_recall_fscore_support(y_real.argmax(axis=1), y_pred.argmax(axis=1), zero_division=0)
        except:
            metricas = precision_recall_fscore_support(y_real, y_pred, zero_division=0)

        # Arranjo do dataframe e inclusão de outras métricas
        metricas_df = pd.DataFrame(list(metricas))
        metricas_df = metricas_df.transpose()

        metricas_df.columns = ['Precision', 'Recall', 'F-score(a=1)', 'Total']
        metricas_df['Accuracy'] = acc
        metricas_df['Class'] = STATUS
        metricas_df['Total'] = metricas_df['Total'].astype(int)

        metricas_df['TP'] = tp.astype(int)
        metricas_df['TN'] = tn.astype(int)
        metricas_df['FP'] = fp.astype(int)
        metricas_df['FN'] = fn.astype(int)

        metricas_df['Specificity'] = (metricas_df['TN']/(metricas_df['TN'] + metricas_df['FP']))
        metricas_df['FPR(FAR)'] = (metricas_df['FP']/(metricas_df['FP'] + metricas_df['TN']))
        metricas_df['F-score(a=0.5)'] =\
            (1.5*metricas_df['Precision']*metricas_df['Recall'])/((0.5*metricas_df['Precision']) + metricas_df['Recall'])
        metricas_df['F-score(a=2)'] =\
            (3*metricas_df['Precision']*metricas_df['Recall'])/((2*metricas_df['Precision']) + metricas_df['Recall'])
        metricas_df['F-score(a=0.5)'].fillna(0, inplace=True)
        metricas_df['F-score(a=2)'].fillna(0, inplace=True)

        metricas_df = metricas_df.set_index('Class')
        metricas_df = metricas_df[['Precision', 'Recall', 'F-score(a=1)', 'F-score(a=0.5)', 'F-score(a=2)', \
                                   'Specificity', 'Accuracy', 'FPR(FAR)', 'TP', 'TN', 'FP', 'FN', 'Total']]
    
    # Classificação binária
    else:
        
        try:
            tn, fp, fn, tp = confusion_matrix(y_real.argmax(axis=1), y_pred.argmax(axis=1)).ravel()
        except:
            tn, fp, fn, tp = confusion_matrix(y_real, y_pred).ravel()
        
        recall = tp/(tp+fn)
        precision = tp/(tp+fp)
        f1_score = 2*recall*precision/(recall+precision)
        f2_score = 3*recall*precision/((2*precision)+recall)
        f05_score = 1.5*recall*precision/((0.5*precision)+recall)
        accuracy = (tp+tn)/(tp+tn+fp+fn)
        specificity = tn/(fp+tn)
        fpr = fp/(fp+tn)
        
        metricas_df = pd.DataFrame([precision, recall, f1_score, f05_score, f2_score, specificity, accuracy, fpr, tp, \
                                    tn, fp, fn]).T
        metricas_df.columns = ['Precision', 'Recall', 'F-score(a=1)', 'F-score(a=0.5)', 'F-score(a=2)', \
                               'Specificity', 'Accuracy', 'FPR(FAR)', 'TP', 'TN', 'FP', 'FN']

    return metricas_df

In [4]:
def display_metrics(x_data, y_data, model, STATUS, set_name, multi_problem):
    
    # Predição usando o modelo treinado
    model_pred = model.predict(x_data)
    try:
        new_status = np.unique(model_pred.argmax(axis=1))
    except:
        new_status = np.unique(model_pred)

    try:
        try:
            df_cm = pd.DataFrame(confusion_matrix(y_data.argmax(axis=1), model_pred.argmax(axis=1)), \
                                 index=[i for i in new_status], columns=[i for i in new_status])
        except:
            df_cm = pd.DataFrame(confusion_matrix(y_data, model_pred, index=[i for i in new_status],
                          columns=[i for i in new_status]))
    except:
        df_cm = pd.DataFrame(confusion_matrix(y_data, model_pred))

    # Linha para normalizar os dados
    df_cm_norm = round((df_cm.astype('float')/df_cm.sum(axis=1).values.reshape(-1,1)), 3)
    
    # Gráfico da matriz de confusão
    plt.figure(figsize=(6,5), dpi=100)
    plt.title("Confusion Matrix - " + set_name + " - CNN", fontsize=10)
    ax = sn.heatmap(df_cm_norm, annot=True, cmap='PuBu')
    ax.set_xlabel("PREDICTED CLASSES", fontsize=8)
    ax.set_ylabel("REAL CLASSES", fontsize=8)
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    plt.show()
    
    # Chama a função metrics()
    data_metrics = metrics(y_data, model_pred, model, df_cm, STATUS, multi_problem)
    
    if (multi_problem):
        pr_micro = precision_score(y_data.argmax(axis=1), model_pred.argmax(axis=1), labels=STATUS, average='micro')
        pr_macro = precision_score(y_data.argmax(axis=1), model_pred.argmax(axis=1), labels=STATUS, average='macro')
        pr_weigh = precision_score(y_data.argmax(axis=1), model_pred.argmax(axis=1), labels=STATUS, average='weighted')

        rc_micro = recall_score(y_data.argmax(axis=1), model_pred.argmax(axis=1), labels=STATUS, average='micro')
        rc_macro = recall_score(y_data.argmax(axis=1), model_pred.argmax(axis=1), labels=STATUS, average='macro')
        rc_weigh = recall_score(y_data.argmax(axis=1), model_pred.argmax(axis=1), labels=STATUS, average='weighted')

        f1_micro = f1_score(y_data.argmax(axis=1), model_pred.argmax(axis=1), labels=STATUS, average='micro')
        f1_macro = f1_score(y_data.argmax(axis=1), model_pred.argmax(axis=1), labels=STATUS, average='macro')
        f1_weigh = f1_score(y_data.argmax(axis=1), model_pred.argmax(axis=1), labels=STATUS, average='weighted')

        print("PRECISION")
        print("micro:    {:.2f}%".format((pr_micro*100)))
        print("macro:    {:.2f}%".format((pr_macro*100)))
        print("weighted: {:.2f}%".format((pr_weigh*100)))

        print("\nRECALL")
        print("micro:    {:.2f}%".format((rc_micro*100)))
        print("macro:    {:.2f}%".format((rc_macro*100)))
        print("weighted: {:.2f}%".format((rc_weigh*100)))

        print("\nF-SCORE (a=1)")
        print("micro:    {:.2f}%".format((f1_micro*100)))
        print("macro:    {:.2f}%".format((f1_macro*100)))
        print("weighted: {:.2f}%".format((f1_weigh*100)))
        print()
        
        try:
            print("Overall AUC:       {:.2f}%".format((roc_auc_score(y_data, model_pred)*100)))
        except:
            pass
        print("Overall Accuracy:    {:.2f}%".format(accuracy_score(y_data.argmax(axis=1), model_pred.argmax(axis=1))*100))
    
    else: 
        print("\nOverall Precision:       {:.2f}%".format((data_metrics['Precision'].values.item()*100)))
        print("Overall Recall:          {:.2f}%".format((data_metrics['Recall'].values.item()*100)))
        print("Overall F-score(a=1):    {:.2f}%".format((data_metrics['F-score(a=1)'].values.item()*100)))
        print("Overall F-score(a=0.5):  {:.2f}%".format((data_metrics['F-score(a=0.5)'].values.item()*100)))
        print("Overall F-score(a=2):    {:.2f}%".format((data_metrics['F-score(a=2)'].values.item()*100)))
        print("Overall Specificity:     {:.2f}%".format((data_metrics['Specificity'].values.item()*100)))
        print("Overall FPR(FAR):        {:.2f}%".format((data_metrics['FPR(FAR)'].values.item()*100)))
        print("Overall Accuracy:        {:.2f}%".format((data_metrics['Accuracy'].values.item()*100)))
    
    return data_metrics  

## Pré-processamento

In [5]:
def corrige_status(status, X_train, X_test, X_valid):
    
    # Cria uma nova coluna auxiliar
    status['soma'] = status['status_duration'].cumsum()
    status['soma'].iloc[0] = 0
    
    # Indexes de "quebra" de acordo com a divisão feita pelo train_test_split
    x_train_end = X_train.index.max()
    x_val_init = x_train_end + 1
    x_val_end = X_valid.index.max()
    x_test_init = x_val_end + 1
    
    # Percorre o banco de parâmetros
    # Precisa encontrar em qual intervalo o index de "quebra" do banco original se encontra
    for i in range(len(status)):
    
        try:
            low_lim = status['soma'].iloc[i]
            upper_lim = status['soma'].iloc[i+1]
        except:
            pass

        if (low_lim <= x_train_end <= upper_lim):
            train_temp = status.iloc[:i+2,:]
            diff_train = x_train_end - train_temp['soma'].iloc[-2] + 1

            status_train = train_temp.copy()
            status_train['status_duration'].iloc[-1] = diff_train
            status_train.drop(['soma'], 1, inplace=True)
            
            train_len = status_train.status_duration.sum()

        if (low_lim <= x_val_end <= upper_lim):
            valid_temp = status.iloc[status_train.index.max():i+2,:]
            diff_valid = x_val_end - valid_temp['soma'].iloc[-2] + 1

            status_valid = valid_temp.copy()
            status_valid['status_duration'].iloc[0] = status_valid['status_duration'].iloc[0] - diff_train
            status_valid['status_duration'].iloc[-1] = diff_valid
            status_valid.drop(['soma'], 1, inplace=True)
            
            valid_len = status_valid.status_duration.sum()

        if (low_lim <= x_test_init <= upper_lim):
            test_temp = status.iloc[status_valid.index.max():,:]

            status_test = test_temp.copy()
            status_test['status_duration'].iloc[0] = status_test['status_duration'].iloc[0] - diff_valid
            status_test.drop(['soma'], 1, inplace=True)
            
            test_len = status_test.status_duration.sum()
    
    # Conferência
    if (status.soma.iloc[-1] == (train_len+valid_len+test_len)):
        print("Divisão dos bancos de dados concluída!")
    else:
        raise ValueError('A divisão dos bancos de dados não foi feita corretamente')
    
    return status_train, status_valid, status_test

In [ ]:
# Para quando se tem apenas treino e teste

def corrige_status_2sets(status, X_train, X_test):
    
    # Cria uma nova coluna auxiliar
    status['soma'] = status['status_duration'].cumsum()
    status['soma'].iloc[0] = 0
    
    # Indexes de "quebra" de acordo com a divisão feita pelo train_test_split
    x_train_end = X_train.index.max()
    x_test_init = x_train_end + 1
    
    # Percorre o banco de parâmetros
    # Precisa encontrar em qual intervalo o index de "quebra" do banco original se encontra
    for i in range(len(status)):
    
        try:
            low_lim = status['soma'].iloc[i]
            upper_lim = status['soma'].iloc[i+1]
        except:
            pass

        if (low_lim <= x_train_end <= upper_lim):
            train_temp = status.iloc[:i+2,:]
            diff_train = x_train_end - train_temp['soma'].iloc[-2] + 1

            status_train = train_temp.copy()
            status_train['status_duration'].iloc[-1] = diff_train
            status_train.drop(['soma'], 1, inplace=True)
            
            train_len = status_train.status_duration.sum()

        if (low_lim <= x_test_init <= upper_lim):
            test_temp = status.iloc[status_train.index.max():,:]

            status_test = test_temp.copy()
            status_test['status_duration'].iloc[0] = status_test['status_duration'].iloc[0] - diff_train
            status_test.drop(['soma'], 1, inplace=True)
            
            test_len = status_test.status_duration.sum()
    
    # Conferência
    if (status.soma.iloc[-1] == (train_len+test_len)):
        print("Divisão dos bancos de dados concluída!")
    else:
        raise ValueError('A divisão dos bancos de dados não foi feita corretamente')
    
    return status_train, status_test

In [6]:
def data_scaling(X_train, X_test, X_valid, scaling_mode):
    
    # Inicializa o dataframe onde as métricas do banco de treino serão armazenadas
    scaling_info = pd.DataFrame(range(len(X_train.columns)), columns=['mean_train'])
    scaling_info['std_train'] = np.zeros(())
    scaling_info['max_train'] = np.zeros(())
    scaling_info['min_train'] = np.zeros(())
    
    i=0
    # Calcula as métricas de cada coluna do banco de treino e armazena no df sacaling_info
    for col in X_train.columns:
    
        col_mean = X_train[col].mean()
        scaling_info['mean_train'].iloc[i] = col_mean

        col_std = X_train[col].std()
        scaling_info['std_train'].iloc[i] = col_std

        col_max = X_train[col].max()
        scaling_info['max_train'].iloc[i] = col_max

        col_min = X_train[col].min()
        scaling_info['min_train'].iloc[i] = col_min

        i+=1
    
    DATASET_DICT = {'train': X_train, 'valid': X_valid, 'test': X_test}
    SCALED_DATASET_DICT = {}
    
    # Percorre cada um bancos previamente divididos e aplica o scaling selecionado
    for key in DATASET_DICT.keys():
        data = DATASET_DICT[key]
        
        i=0
        for col in data.columns:
            
            if (scaling_mode == 'standardizing'):
                data[col] = (data[col]-scaling_info['mean_train'].iloc[i])/scaling_info['std_train'].iloc[i]
            
            if (scaling_mode == 'normalizing_0'):
                data[col] = (data[col]-scaling_info['min_train'].iloc[i])/(scaling_info['max_train'].iloc[i]- \
                                                                           scaling_info['min_train'].iloc[i])
        
            if (scaling_mode == 'normalizing_1'):
                data[col] = (data[col]-scaling_info['min_train'].iloc[i])/(scaling_info['max_train'].iloc[i]- \
                                                                           scaling_info['min_train'].iloc[i])
                data[col] = (data[col]*2) - 1
            
            i+=1
        
        SCALED_DATASET_DICT[key] = data.copy()
    
    print("Scaling dos bancos de dados concluído!")
    print()
    
    return SCALED_DATASET_DICT

In [ ]:
# Para quando se tem apenas treino e teste

def data_scaling_2sets(X_train, X_test, scaling_mode, X_valid=pd.DataFrame()):
    
    # Inicializa o dataframe onde as métricas do banco de treino serão armazenadas
    scaling_info = pd.DataFrame(range(len(X_train.columns)), columns=['mean_train'])
    scaling_info['std_train'] = np.zeros(())
    scaling_info['max_train'] = np.zeros(())
    scaling_info['min_train'] = np.zeros(())
    
    i=0
    # Calcula as métricas de cada coluna do banco de treino e armazena no df sacaling_info
    for col in X_train.columns:
    
        col_mean = X_train[col].mean()
        scaling_info['mean_train'].iloc[i] = col_mean

        col_std = X_train[col].std()
        scaling_info['std_train'].iloc[i] = col_std

        col_max = X_train[col].max()
        scaling_info['max_train'].iloc[i] = col_max

        col_min = X_train[col].min()
        scaling_info['min_train'].iloc[i] = col_min

        i+=1
    
    DATASET_DICT = {'train': X_train, 'valid': X_valid, 'test': X_test}
    SCALED_DATASET_DICT = {}
    
    # Percorre cada um bancos previamente divididos e aplica o scaling selecionado
    for key in DATASET_DICT.keys():
        data = DATASET_DICT[key]
        
        i=0
        for col in data.columns:
            
            if (scaling_mode == 'standardizing'):
                data[col] = (data[col]-scaling_info['mean_train'].iloc[i])/scaling_info['std_train'].iloc[i]
            
            if (scaling_mode == 'normalizing_0'):
                data[col] = (data[col]-scaling_info['min_train'].iloc[i])/(scaling_info['max_train'].iloc[i]- \
                                                                           scaling_info['min_train'].iloc[i])
        
            if (scaling_mode == 'normalizing_1'):
                data[col] = (data[col]-scaling_info['min_train'].iloc[i])/(scaling_info['max_train'].iloc[i]- \
                                                                           scaling_info['min_train'].iloc[i])
                data[col] = (data[col]*2) - 1
            
            i+=1
        
        SCALED_DATASET_DICT[key] = data.copy()
    
    print("Scaling dos bancos de dados concluído!")
    print()
    
    return SCALED_DATASET_DICT

In [ ]:
def vectorized_stride_ana(array, max_time, sub_window_size, stride_size):
    
    sub_windows = ( 
        np.expand_dims(np.arange(sub_window_size), 0) +
        np.expand_dims(np.arange(max_time + 1), 0).T
    )
    
    # Descobre o index da última coluna do array
    last_col_index = (array.shape[1])-1
    
    # Linha da matriz de índices que vai até o tamanho total do trecho que será convertido em matrizes
    cut_point = np.where(sub_windows[:,last_col_index] == len(array)-1)[0].item()
    
    # Faz o corte
    sub_windows_new = sub_windows[:cut_point]
    
    # Fancy indexing to select every V rows.
    return array[sub_windows_new[::stride_size]]

In [ ]:
# Nova estratégia utilizando recursos de indexação do numpy

def matrix_generator(scaled_data, status_data, nlinhas, ncolunas, sliding_window):
    
    x_windows = np.empty(shape=(1, nlinhas, ncolunas, 1))
    y_windows = np.array([])
    
    # Converte o df com dados de entrada em um array
    x_array = scaled_data.values

    c = status_data.iloc[0,1]  # Obtem o tamanho do primeiro dataset
    ct1 = 0
    ct2 = 0
    ct3 = 0 

    for j in range(len(status_data)):

        # Necessário para fazer a primeira concatenação - não é vazio
        x_data = np.empty(shape=(1, nlinhas, ncolunas, 1)) 
        data = x_array[ct1:c+ct2,:] 

        # Necessário para lidar com o fato do primeiro índice em python ser o zero
        if ct3 < len(status_data)-1:
            tamanho = status_data.iloc[ct3+1,1]

        # Atualização dos contadores auxiliares para a próxima iteração
        ct1 = c + ct2
        ct2 = ct2 + tamanho                                                          
        ct3 = ct3 + 1
        
        print("---------------------------------------------------")
        print("Status: ", status_data.iloc[j,0])
        if len(data) < nlinhas:
            print("Status com tamanho menor ao da janela. Dados ignorados!")
        else:
            windows_num = int((len(data)-nlinhas)/sliding_window)+1
            print("Número de janelas: ", windows_num)

            # Chama a função que cria as matrizes para o status em questão e faz o reshape necessário
            res = vectorized_stride_ana(data, len(data)-1, nlinhas, sliding_window)
            x_data_slice_reshape = res.reshape((len(res), nlinhas, ncolunas, 1), order='C')

            # Cria um vetor com os labels de cada matriz
            matrix_len = len(x_data_slice_reshape)
            y_data = np.full((matrix_len), status_data.iloc[j,0])    
            
            # Empilha as matrizes criadas numa única variável
            x_windows = np.concatenate([x_windows, x_data_slice_reshape])
            y_windows = np.append(y_windows, y_data)

    # Remove o primeiro array referente ao np.empty inicial
    x_windows = x_windows[1:, :] 
    
    # Quando nem todos os STATUS estão presentes no banco, 'get_dummies' é uma alternativa
    y_windows_df = pd.DataFrame(y_windows).astype('category')
    y_windows_df_ohe = pd.get_dummies(y_windows_df)
    y_windows_ohe = y_windows_df_ohe.values
    
    return x_windows, y_windows, y_windows_ohe

In [8]:
def data_prepro(
    raw_input, targets, status, nlinhas, ncolunas, sliding_window, test_size, valid_size, scaling_mode):
    
    # Divisão do banco total entre treino, validação e teste
    X_train, X_test, y_train, y_test = train_test_split(raw_input, targets, test_size=test_size, shuffle=False)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=valid_size, shuffle=False)
    
    # Gerando 03 novos dfs com as informações de status e durações corrigidas
    status_train, status_valid, status_test = corrige_status(status, X_train, X_test, X_valid)
    
    # Scaling dos bancos de dados de acordo com o dataset de treinamento
    SCALED_DATASET_DICT = data_scaling(X_train, X_test, X_valid, scaling_mode)
    
    # Gerando as matrizes para cada banco de dados
    x_windows_train, y_windows_train, y_windows_ohe_train =\
        matrix_generator(SCALED_DATASET_DICT['train'], status_train, nlinhas, ncolunas, sliding_window)
    
    x_windows_valid, y_windows_valid, y_windows_ohe_valid =\
        matrix_generator(SCALED_DATASET_DICT['valid'], status_valid, nlinhas, ncolunas, sliding_window)
    
    x_windows_test, y_windows_test, y_windows_ohe_test =\
        matrix_generator(SCALED_DATASET_DICT['test'], status_test, nlinhas, ncolunas, sliding_window)
    
    print("\nReshape dos dados em arrays 4D concluído!")
    
    return x_windows_train, y_windows_train, y_windows_ohe_train, x_windows_valid, y_windows_valid, y_windows_ohe_valid, \
            x_windows_test, y_windows_test, y_windows_ohe_test

In [ ]:
def apply_lag(df, lag=1):
    
    # Aplica a função lagmat do statsmodels
    # 'trim' é o método de corte a ser usado: ‘forward’ corta as observações inválidas na frente
    # 'original' é como o array original é tratado: 'in' retorna o array original e o atrasado como dois arrays separados
    # [lag:,:] desconsidera as linhas nas quais observações foram zeradas - ocorre com a aplicação dos lags
    array_lagged = lagmat(df, maxlag=lag, trim="forward", original='in')[lag:,:]  
    new_columns = []
    
    # Armazena em um array os nomes das novas colunas atrasadas de acordo com quantos lags foram especificados
    for l in range(lag):
        new_columns.append(df.columns+'_lag'+str(l+1))
    
    # Unifica num array os nomes de todas as colunas, originais e atrasadas
    columns_lagged = df.columns.append(new_columns)
    
    # Define os indexes do novo df que será criado
    index_lagged = df.index[lag:]
    
    # Organiza o novo df, com o novo header, novos indexes e os dados originais e atrasados
    df_lagged = pd.DataFrame(array_lagged, index=index_lagged, columns=columns_lagged)
       
    return df_lagged 

In [7]:
# Pouco performático
# Percorrendo o dataframe por um loop e usando loc e iloc

# def matrix_generator(scaled_data, status_data, nlinhas, ncolunas, sliding_window):
    
#     x_windows = np.empty(shape=(1, nlinhas, ncolunas, 1))
#     y_windows = np.array([])

#     c = status_data.iloc[0,1]  # Obtem o tamanho do primeiro dataset
#     ct1 = 0
#     ct2 = 0
#     ct3 = 0 

#     for j in range(len(status_data)):

#         x_data = np.empty(shape=(1, nlinhas, ncolunas, 1)) # Necessário para fazer a primeira concatenação - não é vazio
#         data = scaled_data.iloc[ct1:c+ct2,:] 

#         # Necessário para lidar com o fato do primeiro índice em python ser o zero
#         if ct3 < len(status_data)-1:
#             tamanho = status_data.iloc[ct3+1,1]

#         # Atualização dos contadores auxiliares para a próxima iteração
#         ct1 = c + ct2
#         ct2 = ct2 + tamanho                                                          
#         ct3 = ct3 + 1

#         # Informa se é possível construir uma janela de dados com o trecho do banco selecionado
#         if len(data) < nlinhas:
#             print("Status com tamanho menor ao da janela. Dados ignorados!")
#         else:
#             windows_num = int((len(data)-nlinhas)/sliding_window)+1
#             print("Número de janelas: ", windows_num)

#             y_data = np.empty((windows_num, 1))

#             for i in range(0, windows_num):
#                 x_data_slice = data.iloc[(i*sliding_window):((i*sliding_window)+nlinhas), :]
#                 x_data_slice_reshape = x_data_slice.values.reshape((1, nlinhas, ncolunas, 1), order='C')
#                 x_data = np.concatenate([x_data, x_data_slice_reshape]) 
#                 y_data[i] = status_data.iloc[j,0]

#             x_data = x_data[1:, :] # Remove o primeiro array referente ao np.empty inicial
#             x_windows = np.concatenate([x_windows, x_data])
#             y_windows = np.append(y_windows, y_data)        

#     x_windows = x_windows[1:, :] # Remove o primeiro array referente ao np.empty inicial
    
#     # Quando nem todos os STATUS estão presentes no banco, 'get_dummies' é uma alternativa
#     y_windows_df = pd.DataFrame(y_windows).astype('category')
#     y_windows_df_ohe = pd.get_dummies(y_windows_df)
#     y_windows_ohe = y_windows_df_ohe.values
    
#     return x_windows, y_windows, y_windows_ohe

In [9]:
# Quando data leaking estava ocorrendo

# def data_prepro_antigo(raw_data, sim_status, nlinhas, ncolunas, sliding_window, standardization=False):
    
#     if (standardization == False):
#         min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
#         scaled = min_max_scaler.fit_transform(raw_data)
#         data_total_norm = pd.DataFrame(scaled, index=raw_data.index, columns=raw_data.columns)
#     else:
#         scaled = preprocessing.scale(raw_data)
#         data_total_norm = pd.DataFrame(scaled, index=raw_data.index, columns=raw_data.columns)
    
#     x_windows = np.empty(shape=(1, nlinhas, ncolunas, 1))
#     y_windows = np.array([])

#     c = sim_status.iloc[0,1]  # Obtem o tamanho do primeiro dataset
#     ct1 = 0
#     ct2 = 0
#     ct3 = 0 

#     for j in range(len(sim_status)):

#         x_data = np.empty(shape=(1, nlinhas, ncolunas, 1)) # Necessário para fazer a primeira concatenação - não é vazio
#         data = data_total_norm.iloc[ct1:c+ct2,:] # não teria que ter um iloc?

#         # Necessário para lidar com o fato do primeiro índice em python ser o zero
#         if ct3 < len(sim_status)-1:
#             tamanho = sim_status.iloc[ct3+1,1]

#         # Atualização dos contadores auxiliares para a próxima iteração
#         ct1 = c + ct2
#         ct2 = ct2 + tamanho                                                          
#         ct3 = ct3 + 1

#         if len(data) < nlinhas:
#             print("Status com tamanho menor ao da janela. Dados ignorados!")
#         else:
#             windows_num = int((len(data)-nlinhas)/sliding_window)+1
#             print("Número de janelas: ", windows_num)

#             y_data = np.empty((windows_num, 1))

#             for i in range(0, windows_num):
#                 x_data_slice = data.iloc[(i*sliding_window):((i*sliding_window)+nlinhas), :]
#                 x_data_slice_reshape = x_data_slice.values.reshape((1, nlinhas, ncolunas, 1), order='C')
#                 x_data = np.concatenate([x_data, x_data_slice_reshape]) 
#                 y_data[i] = sim_status.iloc[j,0]

#             x_data = x_data[1:, :] # Remove o primeiro array referente ao np.empty inicial
#             x_windows = np.concatenate([x_windows, x_data])
#             y_windows = np.append(y_windows, y_data)        

#     x_windows = x_windows[1:, :] # Remove o primeiro array referente ao np.empty inicial
    
#     # Quando nem todos os STATUS estão presentes no banco, 'get_dummies' é uma alternativa
#     y_windows_df = pd.DataFrame(y_windows).astype('category')
#     y_windows_df_ohe = pd.get_dummies(y_windows_df)
#     y_windows_ohe = y_windows_df_ohe.values
    
#     return x_windows, y_windows, y_windows_ohe